In [1]:
#Part 2

In [2]:
##from object dump file this is the info for "puts"  
##11d4:	e8 97 fe ff ff       	callq  1070 <puts@plt>
putsID = "1070"

In [3]:
import sys
import angr

In [4]:
proj = angr.Project("./test", load_options={'auto_load_libs': False})
print(proj)

WARNING | 2022-11-18 14:14:08,231 | cle.loader | The main binary is a position-independent executable. It is being loaded with a base address of 0x400000.


<Project ./test>


In [5]:
cfg = proj.analyses.CFGFast(data_references=True, normalize=True)
print(cfg.graph)

DiGraph with 67 nodes and 78 edges


In [6]:
nodelist = list(cfg.graph.nodes())
#print(nodelist)
#Make a list to hold all of the nodes detected

numofnodes = len(nodelist)
print(numofnodes)
#print the total num of nodes

67


In [7]:
edgelist = list(cfg.graph.edges())
#print(edgelist)
#Make a list to hold all of the edges detected

numofedges = len(edgelist)
print(numofedges)
#print the total num of edges

78


In [9]:
for node in nodelist:
    if node.block is None:
        continue
    for insn in node.block.capstone.insns:
        mne = insn.mnemonic
        if mne == 'call':
            if insn.op_str.endswith(putsID):
                addr_target = insn.address
                
                print(hex(addr_target))
                
                initial_state = proj.factory.entry_state()
                simulation = proj.factory.simgr(initial_state)
                print(initial_state)
                
                simulation.explore(find=addr_target)
                
                if simulation.found:
                    solution_state = simulation.found[0]
                    solution = solution_state.posix.dumps(sys.stdin.fileno())
                    print("Success! Solution is: {}".format(solution.decode("utf-8")))
                
                else:
                    raise Exception("Could not find the solution")
                    

0x4011d4
<SimState @ 0x4010a0>


WARNING | 2022-11-18 14:20:31,488 | angr.storage.memory_mixins.default_filler_mixin | Filling memory at 0x7fffffffffeff9c with 4 unconstrained bytes referenced from 0x4010a9 (_start+0x9 in test (0x10a9))
WARNING | 2022-11-18 14:20:32,264 | angr.storage.memory_mixins.default_filler_mixin | Filling memory at 0x7fffffffffeff9c with 4 unconstrained bytes referenced from 0x4010a9 (_start+0x9 in test (0x10a9))


Success! Solution is: 0000003214
0x4011e2
<SimState @ 0x4010a0>
Success! Solution is: 2181088768
